In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

creating the sqlalchemy engine to connect to the database

In [2]:
load_dotenv()
password = os.getenv("DATABASE_PASSWORD")
db_name = os.getenv("DATABASE_NAME")
port = os.getenv("DATABASE_PORT")
hostname = os.getenv("DATABASE_HOSTNAME")
username = os.getenv("DATABASE_USERNAME")

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{db_name}")

Reading in the data

In [3]:
imd2007 = pd.read_csv("new_data/imd2007_england_msoa11.csv")
imd2019 = pd.read_csv("new_data/imd2019_msoa_level_data.csv")

imd2007.head()

,msoa11_code,Score,Proportion,Extent
0,E02000001,13.111167,0.0,0.029886
1,E02000002,40.145799,0.5,0.716106
2,E02000003,24.074696,0.0,0.033752
3,E02000004,24.142289,0.0,0.116677
4,E02000005,30.463064,0.0,0.285928


In [4]:
imd2019.head()

,MSOAC,MSOAHOCLN,LAD19C,LAD19N,REG,LSOACOUNT,POPMID15,IMD19 SCORE,MSOARANK,MSOADECILE,MSOAQUINTILE
0,E02000001,City of London,E09000001,City of London,London,6,6687,14.720484,4198,7,4
1,E02000002,Marks Gate,E09000002,Barking and Dagenham,London,4,7379,34.767560,1060,2,1
2,E02000003,Chadwell Heath East,E09000002,Barking and Dagenham,London,6,10720,25.149726,2200,4,2
3,E02000004,Eastbrookend,E09000002,Barking and Dagenham,London,4,6536,23.560044,2429,4,2
4,E02000005,Becontree Heath,E09000002,Barking and Dagenham,London,5,9243,30.212965,1519,3,2


Reformatting the column headers so they are SQL-friendly

In [6]:
for df in [imd2007, imd2019]:
    df.columns = map(str.lower, df.columns)
    df.columns = map(lambda s: s.replace(" ", "_"), df.columns)

imd2019.head()

,msoac,msoahocln,lad19c,lad19n,reg,lsoacount,popmid15,imd19_score,msoarank,msoadecile,msoaquintile
0,E02000001,City of London,E09000001,City of London,London,6,6687,14.720484,4198,7,4
1,E02000002,Marks Gate,E09000002,Barking and Dagenham,London,4,7379,34.767560,1060,2,1
2,E02000003,Chadwell Heath East,E09000002,Barking and Dagenham,London,6,10720,25.149726,2200,4,2
3,E02000004,Eastbrookend,E09000002,Barking and Dagenham,London,4,6536,23.560044,2429,4,2
4,E02000005,Becontree Heath,E09000002,Barking and Dagenham,London,5,9243,30.212965,1519,3,2


Then simply uploading the data to SQL

In [7]:
imd2007.to_sql("imd2007", con=engine, if_exists='replace', index=False)
imd2019.to_sql("imd2019", con=engine, if_exists='replace', index=False)

791